# Part 2 - Price prediction using Natural Language Processing (NLP)

In Part 1, I used attributes of Les Paul models to predict the retail price. This approach worked in principle but the model error was too large to be useful. A better approach might be to use the product description to identify key pieces of text which would allude to the price. For example, a basic guitar might have a very simplistic product description but an expensive guitar could have some unique keywords which would allude to a higher price tag. With this in mind, let's try NLP to predict guitar prices!